In [ ]:

import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss, acf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import boxcox
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from scipy.stats import ttest_rel

In [ ]:
from src import utils
from src import validation
from src import base_forecast
from src.utils import print_title

In [ ]:
import importlib
importlib.reload(utils)
importlib.reload(validation)
importlib.reload(base_forecast)

## DATA INGESTION

### LOAD

In [ ]:
day_df_raw = pd.read_csv('../data/day.csv')
display(day_df_raw.head())
print('DF Size: ', day_df_raw.shape)
print('DF Types: \n', day_df_raw.dtypes)

df = day_df_raw.copy()

### DTYPES

In [ ]:
datetime_columns = ['dteday']
float64_columns = ['temp','atemp','hum','windspeed']
str_columns = ['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']
int64_columns = ['casual', 'registered', 'cnt']

# Basic data conversion
df = utils.format_columns(df, datetime_columns, int64_columns, float64_columns, str_columns)
# Rename columns
df.rename(columns={
    'dteday':'date',
    'yr':'year',
    'mnth':'month',
    'weathersit':'weather',
    'temp':'temperature',
    'atemp':'temperature_sensation',
    'hum':'humidity',
    'casual':'casual_users',
    'registered':'registered_users',
    'cnt':'bikes_rented'
}, inplace=True)
# Drop not used columns
df.drop(columns=['instant'], inplace=True)

# Check dtypes
print_title('CONVERTED DATA TYPES')
print(df.dtypes)
display(df.head(5))

## DATA CLEANING AND QA

### DESCRIBE

In [ ]:
# Quick checks on data
print_title('DF INFO')
display(df.info())

print_title('DF DESCRIBE')
display(df.describe())

# Check distribution of variants
print_title('DISTRIBUTIONS')
display(df['season'].value_counts().sort_index())
display(df['year'].value_counts().sort_index())
display(df['month'].value_counts().sort_index())
display(df['holiday'].value_counts().sort_index())
display(df['weekday'].value_counts().sort_index())
display(df['workingday'].value_counts().sort_index())
display(df['weather'].value_counts().sort_index())

### DUPLICATES

In [ ]:
# Check for and drop duplicates in the entire DataFrame
duplicated_rows = df.duplicated().sum()
print('# of duplicated rows: ', duplicated_rows)

if duplicated_rows > 0:
    df = df.drop_duplicates()
    print('Duplicates in the DataFrame removed.')
else:
    print('No duplicates in the DataFrame found.')

In [ ]:

primary_key_column = 'date'

# Check for duplicates in the unique columns
duplicated_rows = df[df[primary_key_column].duplicated(keep=False)]
print(f'# of duplicated on {primary_key_column} column: {duplicated_rows[primary_key_column].nunique()}')

if not duplicated_rows.empty:
    print(f'Duplicated {primary_key_column} and their rows:')
    display(duplicated_rows.sort_values(by = primary_key_column))

    # Keep only the first following timestamp column order
    if primary_key_column == '':
        df = df.drop_duplicates(subset=primary_key_column, keep='first')
        print('Kept the most recent row for each duplicated' +  primary_key_column)
    else:
        df = df.sort_values(primary_key_column).drop_duplicates(subset=primary_key_column, keep='first')
        print('Kept the most recent row for each duplicated ' + primary_key_column)

### NULLS

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print_title('NUMBER OF NULL VALUES')
print(missing_values)

### OUTLIERS

In [ ]:
target_columns = ['casual_users', 'registered_users', 'bikes_rented']

numeric_cols = df.drop(columns=target_columns, errors='ignore').select_dtypes(include=["number"])
outliers_df = utils.detect_outliers(numeric_cols, method="iqr")
outlier_rows = df.loc[outliers_df.any(axis=1)]
print_title('ANOMALY ROWS')
display(outliers_df[outliers_df['is_outlier']])

df["extreme_weather"] = outliers_df["is_outlier"].astype(int).astype(str)
display(df.head())

## EDA

In [ ]:
target_column = 'bikes_rented'
try:
    df.drop(columns=['registered_users', 'casual_users'], inplace= True)
except:
    pass
display(df.head())

In [ ]:
utils.plot_time_series(df, [target_column], 0.9, '--')

### STATIONARITY

In [ ]:

stationary_result = adfuller(df["bikes_rented"])
is_stationary_resultt = True if stationary_result[1] < 0.05 else False
print(f"ADF Statistic: {stationary_result[0]}")
print(f"P-value: {stationary_result[1]}")
print("Conclusion:", "Stationary" if stationary_result[1] < 0.05 else "Non-stationary")

### TREND

In [ ]:

trend_result = kpss(df["bikes_rented"], regression="c")
is_trend_present = True if trend_result[1] < 0.05 else False
print(f"KPSS Statistic: {trend_result[0]}")
print(f"P-value: {trend_result[1]}")
print("Conclusion:", "Trend present" if trend_result[1] < 0.05 else "No significant trend")

### TIME PERSISTENCE AND SEASONALITY

In [ ]:
nlags = 365
acf_values = acf(df["bikes_rented"], nlags=nlags, fft=True)

autocorrelated_lags = np.where(np.abs(acf_values) > 0.5)[0]
autocorrelation_ratio = len(autocorrelated_lags) / nlags

seasonal_lags = [lag for lag in autocorrelated_lags if lag % 7 == 0 or lag % 30 == 0 or lag % 365 == 0]

print(f"Lags with autocorrelation: {autocorrelated_lags.tolist()}")
print(f"Percentage of significant lags: {autocorrelation_ratio:.2%}")
print(f"Possible seasonal lags: {seasonal_lags}")

if autocorrelation_ratio > 0.5:
    print("Strong temporal dependence detected.")
elif autocorrelation_ratio > 0.3:
    print("Moderate temporal dependence detected.")
else:
    print("Low temporal dependence.")

if len(seasonal_lags) > 2:
    is_seasonality_present = True
    print("Seasonality detected.")
else:
    is_seasonality_present = False
    print("No clear seasonality.")

### ACF AND PACF

In [ ]:
utils.plot_acf_and_pacf(df[target_column])

### DECOMPOSITION

In [ ]:
decomposed = seasonal_decompose(df[target_column], model='additive', period=30)

fig, axes = plt.subplots(4, 1, figsize=(16, 16))

axes[0].plot(df["date"], decomposed.observed, color="black", linewidth=2)
axes[0].set_title("Original", fontsize=14, fontweight='bold')

axes[1].plot(df["date"], decomposed.trend, color="tab:blue", linewidth=2)
axes[1].set_title("Trend", fontsize=14, fontweight='bold')

axes[2].plot(df["date"], decomposed.seasonal, color="tab:green", linewidth=2)
axes[2].set_title("Seasonality", fontsize=14, fontweight='bold')

axes[3].plot(df["date"], decomposed.resid, color="tab:red", linewidth=2)
axes[3].set_title("Residuals", fontsize=14, fontweight='bold')

for ax in axes:
    for spine in ax.spines.values():
        spine.set_visible(False)
    ax.grid(True, linestyle="--", alpha=0.5)
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
    ax.tick_params(axis='x', rotation=0, labelsize=10)

plt.tight_layout()
plt.show()

utils.plot_acf_and_pacf(decomposed.resid.dropna(), ' - Residuals')

### OUTLIERS

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))

sns.boxplot(
    x=df[target_column], 
    ax=ax, 
    flierprops={"marker": "o", "markerfacecolor": "red", "markeredgecolor": "black", "markersize": 6}
)

ax.set_title(f"Outlier Detection in {target_column}", fontsize=14, fontweight="bold")
ax.set_xlabel(target_column, fontsize=12, fontweight="bold")

for spine in ax.spines.values():
    spine.set_visible(False)

ax.grid(axis="x", linestyle="--", alpha=0.5)

plt.show()

### EXOGENS

In [ ]:
excluded_columns = ["bikes_rented", "date"]

exogen_columns = [
    col for col in df.columns 
    if col not in excluded_columns and df[col].dtype in ["int64", "float64", "uint8", "object"]
]
print("Exogenous variables for forecasting:", exogen_columns)

### NUMERIC CORRELATION AND COLLINEARITY

In [ ]:
plt.figure(figsize=(6, 2))
numeric_to_corr_df = df.select_dtypes(include=["number"]).drop(columns=[target_column], errors="ignore")
corr_df = numeric_to_corr_df.corr()
sns.heatmap(corr_df, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Correlation between numeric variables")
plt.show()

In [ ]:
X = numeric_to_corr_df.dropna().select_dtypes(include=["number"]).astype("float64")

X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.dropna(inplace=True)

corr_matrix = X.corr().abs()
high_corr_pairs = np.where(corr_matrix > 0.8)
high_corr_pairs = [(corr_matrix.index[i], corr_matrix.columns[j]) 
                   for i, j in zip(*high_corr_pairs) if i != j and i < j]

print("Highly correlated variable pairs:", high_corr_pairs)

vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data = vif_data.sort_values(by="VIF", ascending=False)

print('\n')
print("VIF:")
print(vif_data)

target_corr = df.corr()["bikes_rented"].drop(["bikes_rented", "date"])
print('\n')
print("ABS Correlation values:")
print(abs(target_corr).sort_values(ascending=False))

## BASE FORECAST FEATURE ENGINEERING

### FEATURE SELECTION

In [ ]:
features_to_remove = set()
for pair in high_corr_pairs:
    feature_to_remove = pair[0] if abs(target_corr[pair[0]]) < abs(target_corr[pair[1]]) else pair[1]
    features_to_remove.add(feature_to_remove)

# Remove high VIF features (threshold >10)
high_vif_features = vif_data[vif_data["VIF"] > 10]["Variable"].tolist()
features_to_remove.update(high_vif_features)

# Select relevant features
selected_features = [feature for feature in exogen_columns if feature not in features_to_remove]
selected_features = sorted(selected_features, key=lambda x: abs(target_corr[x]), reverse=True)
print("Final selected exogenous variables:", selected_features)

### TIME NORMALIZATION

In [ ]:
use_time_normalization = False
selection_df = df.copy()
target_time_norm_column = target_column + "_time_norm"

if use_time_normalization:
    forecast_column = target_time_norm_column
    selection_df[forecast_column] = df[target_column] / df["date"].dt.days_in_month
    try:
        selection_df.drop(columns=target_column, inplace=True)
    except:
        pass
else:
    forecast_column = target_column
    try:
        selection_df.drop(columns=target_time_norm_column, inplace=True)
    except:
        pass

print('Forecast variable: ', forecast_column)
display(selection_df.head(1))

### TRANSFORMATIONS

In [ ]:
selection_df = selection_df[['date'] + selected_features + [forecast_column]]

print_title('Recommended Transformations')
transformations = utils.check_transformations(selection_df)

print('\n')
print_title('DF Prepared')
transformed_df = utils.apply_transformations(selection_df, transformations)
print('Forecast variable: ', forecast_column)

## BASELINE AND VALIDATION

### BASELINE

In [ ]:
naive_forecast_df = base_forecast.naive_forecast(transformed_df, forecast_column, steps=60)
baseline_column = 'naive_forecast'

### VALIDATION

In [ ]:
validation.plot_time_series_forecast(naive_forecast_df, [forecast_column, 'naive_forecast'])

#### METRICS

In [ ]:
model_metrics = validation.calculate_forecast_metrics(naive_forecast_df, naive_forecast_df, forecast_column, "naive_forecast")
print(model_metrics)

#### TIME VALIDATION

In [ ]:
# Walk-Forward Validation
walk_results = validation.walk_forward_validation(naive_forecast_df, naive_forecast_df, "bikes_rented", "naive_forecast", steps=30, n_splits=5)

# Expanding Window Validation
expanding_results = validation.expanding_window_validation(naive_forecast_df, naive_forecast_df, "bikes_rented", "naive_forecast", steps=30, initial_train_size=100)

# Rolling Window Validation
rolling_results = validation.rolling_window_validation(naive_forecast_df, naive_forecast_df, "bikes_rented", "naive_forecast", steps=30, window_size=100)

In [ ]:
validation.plot_validation_results(walk_results, expanding_results, rolling_results)

#### RESIDUALS

In [ ]:
validation.check_forecast_residuals(naive_forecast_df, "bikes_rented", "naive_forecast")


### BASELINE COMPARISON

In [ ]:
comparison_results = validation.compare_forecast_models(naive_forecast_df, naive_forecast_df, naive_forecast_df, forecast_column, "naive_forecast")
formatted_results_df = validation.format_comparison_results(comparison_results)
display(formatted_results_df)

## BASE FORECASTING

#### RANDOM WALK

In [ ]:
random_walk_drift = True if is_trend_present else False
random_walk_forecast_df = base_forecast.random_walk_forecast(transformed_df, forecast_column, steps=60, drift=random_walk_drift)
random_walk_forecast_metrics, random_walk_forecast_comparison_results = validation.validate_forecast(naive_df=naive_forecast_df, forecast_df=random_walk_forecast_df, baseline_df=naive_forecast_df, to_forecast_column = forecast_column, forecasted_column = 'random_walk_forecast')

#### EXPONENTIAL SMOOTHING

In [ ]:
exponential_smoothing_forecast_df = exponential_smoothing_forecast(transformed_df, forecast_column, steps=60)
exponential_smoothing_walk_forecast_metrics, exponential_smoothing_comparison_results = validation.validate_forecast(naive_df=naive_forecast_df, forecast_df=exponential_smoothing_forecast_df, baseline_df=naive_forecast_df, to_forecast_column = forecast_column, forecasted_column = 'exponential_smoothing_forecast')

## STATISTICAL FORECASTING

## ML FORECAST FEATURE ENGINEERING

#### COMPLETE CORRELATION AND COLLINEARITY

In [ ]:
plt.figure(figsize=(20, 16))
df_encoded_to_corr = pd.get_dummies(df.drop(columns=[target_column, "date"], errors="ignore"), drop_first=True)
df_corr = df_encoded_to_corr.corr()
sns.heatmap(df_corr, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Correlation between all variables")
plt.show()